# Arrhythmia detection

## ECG signal 
![alt text](ecg.png "Title")

## Packages

In [1]:
import logging
import os
import pickle

import neurokit2 as nk
import numpy as np
import wfdb

from util import map_peak_annotations

## Load the database

In [ ]:
dbdir = 'mit-bih-arrhythmia-database-1.0.0'
if not os.path.isdir(dbdir):
    logging.error('Please replace dbdir with the db directory')
    raise FileNotFoundError(dbdir)

## Constants

In [ ]:
# From the BIH database docs
beat_annotations = ['N', 'L', 'R', 'B', 'A', 'a', 'J', 'S', 'V', 'r', 'F', 'e', 'j', 'n', 'E', '/', 'f', 'Q', '?']
sampling_rate = 360

## Load records, detect peaks, onsets, offsets and map annotations to R peaks

In [ ]:
records = []
for _ in range(100, 220):
    try:
        bih_record = wfdb.rdrecord('{}/{}'.format(dbdir, _))
        annotation = wfdb.rdann('{}/{}'.format(dbdir, _), 'atr')
    except FileNotFoundError:
        continue

    # Create a list of tuples of beat symbols and the sample at which it is annotated 
    cleaned_annotation = np.unique(annotation.sample[np.isin(annotation.symbol, beat_annotations)])
    dtype = [('sample', 'i4'), ('symbol', 'U1')]
    annotations = np.array([(int(i), s) for i, s in zip(cleaned_annotation, annotation.symbol)], dtype=dtype)
    # Read the signal from the bih_record
    ecg_signal = bih_record.p_signal[:, 0]
    # Find all r peaks in a record
    _, r_peaks = nk.ecg_peaks(ecg_signal, sampling_rate=sampling_rate)
    # Map beat annotations to R-peaks
    annotated_r_peaks = map_peak_annotations(r_peaks['ECG_R_Peaks'], annotations)
    # Find other peaks, onsets and offsets
    _, waves_peaks = nk.ecg_delineate(ecg_signal, r_peaks, sampling_rate, method="dwt", show_type='all')
    records.append({'ecg_signal': ecg_signal, 'annotated_r_peaks': annotated_r_peaks, 'waves_peaks': waves_peaks})

## Save records into a file

In [ ]:
with open('records.pkl', 'wb') as file:
    pickle.dump(records, file)

## Load records from a file

In [2]:
with open('records.pkl', 'rb') as file:
    records = pickle.load(file)

KeyboardInterrupt: 

## Distribution of classes

In [5]:
# TODO Adjust to new way of data preprocessing
# labels, values = zip(*Counter([s[0] for s in slices]).items())
# plt.bar(labels, values)
# for i, v in enumerate(values):
#     plt.text(i, v + 0.1, str(v), ha='center', va='bottom')
# plt.title('Distribution of Beat and Non-beat annotations')
# plt.xlabel('Annotation')
# plt.ylabel('Frequency')
# plt.show()

## Map annotations to R waves
N	--	Normal beat (displayed as "·" by the PhysioBank ATM, LightWAVE, pschart, and psfd)

L	--	Left bundle branch block beat

R	--	Right bundle branch block beat

B	--	Bundle branch block beat (unspecified)

A	--	Atrial premature beat

a	--	Aberrated atrial premature beat

J	--	Nodal (junctional) premature beat

S	--	Supraventricular premature or ectopic beat (atrial or nodal)

V	--	Premature ventricular contraction

r	--	R-on-T premature ventricular contraction

F	--	Fusion of ventricular and normal beat

e	--	Atrial escape beat

j	--	Nodal (junctional) escape beat

n	--	Supraventricular escape beat (atrial or nodal)

E	--	Ventricular escape beat

/	--	Paced beat

f	--	Fusion of paced and normal beat

Q	--	Unclassifiable beat

?	--	Beat not classified during learning


## Features for classification:
* age
* sex
* PR interval
* RT interval

## SVM model training - SVM Gaussian radial basis function